**Импортируем необходимые библиотеки**

In [118]:
import requests
import xlsxwriter
import pandas as pd
import numpy as np
import sqlite3
from bs4 import BeautifulSoup as bs

Передаем сайту "User-agent":

In [119]:
headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
          'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

Прописываем url с которого начнем парсинг. В поле 'text=' указываем требуемую вакансию. На сайте hh.ru можно настроить дополнительные фильтры поиска, после этого вставить полученную ссылку сюда:

In [120]:
base_url = 'https://hh.ru/search/resume?L_is_autosearch=false&area=113&clusters=true&exp_period=all_time&logic=phrase&no_magic=false&order_by=relevance&pos=position%2Cworkplace_position&text=менеджер+по+продажам&page='

Функция парсинга: получаем список url с резюме, в которых указана желаемая зарплата, затем собираем данные внутри каждого резюме:

In [121]:
def hh_pars(base_url, headers, pages):
    data = {}
    job_title_data = []
    salary_data = []
    gender_data = []
    age_data = []
    address_data = []
    job_experience_data = []
    employers_data = []
    employers_data_summery = []
    key_skills_data = []
    key_skills_data_summery = []
    education_data = []
    education_data_summery = []
    languages_data = []
    languages_data_summery = []
    
    for page in range(0,pages):
        base_url_2 = base_url + str(page) 
        #print(base_url_2)
        urls = []
        session = requests.Session()
        request = session.get(base_url_2, headers=headers)

        #Getting a list of urls with CV
        if request.status_code == 200:
            soup = bs(request.content, 'html.parser')
            divs = soup.find_all('div', attrs = {'itemscope': 'itemscope'})
            for div in divs:
                compensation = div.find('div', attrs= {'class': 'resume-search-item__compensation'}).text
                #Getting CV only with Salory
                if compensation != '':
                    try:
                        href = div.find('a', attrs= {'data-qa': 'resume-serp__resume-title'})['href']
                        href_full =  'https://hh.ru'+ href
                        urls.append(href_full)
                    except:
                        None
            #print('OK')
        else:
            print('ERROR')

        #Parsing inside urls
        for url in urls:
            session_url = requests.Session()
            request_url = session.get(url, headers=headers)
            if request_url.status_code == 200:
                soup_inside_urls = bs(request_url.content, 'html.parser')

                #job title
                job_title_searching = soup_inside_urls.find('h1', attrs = {'class': 'header'})
                try:
                    job = job_title_searching.find('span').text
                except:
                    job = 'не указано'
                job_title_data.append(job)

                #Desired Salary
                salary_header = soup_inside_urls.find('div', attrs = {'class': 'resume-block-container'})
                salary = salary_header.find('span', attrs = {'data-qa': 'resume-block-salary'}).text
                salary_data.append(salary)

                #Gender
                blok_main_info = soup_inside_urls.find('div', attrs = {'class': 'resume-header'})
                gender = blok_main_info.find('span', attrs = {'itemprop': 'gender'}).text
                gender_data.append(gender)
                try:
                    age = blok_main_info.find('span', attrs = {'data-qa': 'resume-personal-age'}).text
                except:
                    age = np.NaN
                age_data.append(age)

                #Location
                address = blok_main_info.find('span', attrs = {'data-qa': 'resume-personal-address'}).text
                address_data.append(str.upper(address))

                #Experience
                blok_job_experience=soup_inside_urls.find('div', attrs = {'data-qa': 'resume-block-experience'}) 
                try:
                    job_experience = blok_job_experience.find('span', attrs = {'class': 'resume-block__title-text resume-block__title-text_sub'}).text
                except:
                    job_experience = np.NaN
                job_experience_data.append(job_experience)

                #Previous employers
                job_company=soup_inside_urls.find_all('div', attrs = {'itemtype': 'http://schema.org/Organization'})
                for jobs_comp in job_company:
                    try:
                        employers=jobs_comp.find('div', attrs = {'class': 'resume-block__sub-title'}).text
                        employers_data.append(str.upper(employers))
                    except:
                        employers = np.NaN
                employers_data_summery.append(employers_data)
                employers_data = []

                #Key skills
                blok_skills=soup_inside_urls.find_all('div', attrs = {'class': 'bloko-tag bloko-tag_inline bloko-tag_countable'}) 
                if blok_skills != []:
                    for key in blok_skills:
                        try:
                            key_skills = key.find('span').text
                            key_skills_data.append(str.upper(key_skills))
                        except:
                            key_skills = np.NaN
                else:
                    key_skills_data = np.NaN
                key_skills_data_summery.append(key_skills_data)
                key_skills_data = []

                #Education
                blok_education=soup_inside_urls.find_all('div', attrs = {'data-qa': 'resume-block-education-name'}) 
                for edu in blok_education:
                    try:
                        education=edu.text
                    except:
                        education = ''
                    education_data.append(str.upper(education))
                    #print(education)
                education_data_summery.append(education_data)
                education_data = []

                #Languages
                blok_languages=soup_inside_urls.find_all('p', attrs = {'data-qa': 'resume-block-language-item'}) 
                for lang in blok_languages:
                    languages=lang.contents[0]
                    try:
                        #languages_in_ru = city_translator(languages)
                        languages_data.append(str.upper(languages))
                    except:
                        languages_data = np.nan
                    #print(languages)
                languages_data_summery.append(languages_data)
                languages_data = []

                #Getting final dataset in pandas format
                data = {'Желаемая вакансия': job_title_data,
                                         'Желаемая зарплата': salary_data, 
                                         'Пол': gender_data, 
                                         'Возраст': age_data, 
                                         'Местонахождения': address_data, 
                                         'Общий стаж': job_experience_data, 
                                         'Работодатели':  employers_data_summery,
                                         'Ключевые навыки': key_skills_data_summery,
                                         'Образование': education_data_summery, 
                                         'Знание языков': languages_data_summery}    

                #Inputing list in Pandas cells
                data_pd = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()]))
               
            else:
                print('Error')
        print('.', end='')
    
    print('\n','Parsing was completed')
    return data_pd

Вызываем функцию парсинга: передаем ссылку, данные "user - agent", общее количество страниц для парсинга:

In [127]:
data_table=hh_pars(base_url, headers, 50)

..................................................
 Parsing was completed


In [128]:
data_table

,Желаемая вакансия,Желаемая зарплата,Пол,Возраст,Местонахождения,Общий стаж,Работодатели,Ключевые навыки,Образование,Знание языков
0,Менеджер по продажам,80 000 руб.,Мужчина,35 лет,МОСКВА,Опыт работы 14 лет,"[СТИЛЬНЫЙ ПОЛ ООО, АЛИКС ГРУПП, ТОПВУД, ГК ""РО...","[ВЕДЕНИЕ КЛИЕНТСКОЙ БАЗЫ, РОЗНИЧНАЯ ТОРГОВЛЯ, ...","[МОСКОВСКИЙ ГОРОДСКОЙ ИНСТИТУТ МЕНЕДЖМЕНТА, МО...","[РУССКИЙ, АНГЛИЙСКИЙ]"
1,Менеджер по продажам,40 000 руб.,Женщина,20 лет,САНКТ-ПЕТЕРБУРГ,Опыт работы 10 месяцев,"[EXCEED TEAM, GO!ENGLISH ]","[ПОИСК И ПРИВЛЕЧЕНИЕ КЛИЕНТОВ, РЕГИОНАЛЬНЫЕ СЕ...",[ДОНСКОЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕРСИТ...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
2,Менеджер по продажам,35 000 руб.,Женщина,22 года,РОСТОВ-НА-ДОНУ,Опыт работы 2 года 4 месяца,"[АО ""ПФ ""СКБ КОНТУР"", DVA.EXPERT, ИП МАКЕЕВА Е...","[ОПЫТ ПРОДАЖ, ПРЯМЫЕ ПРОДАЖИ, ПОИСК И ПРИВЛЕЧЕ...",[ДОНСКОЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕРСИТ...,[РУССКИЙ]
3,Менеджер по продажам,90 000 руб.,Мужчина,22 года,ОДИНЦОВО,Опыт работы 1 год 5 месяцев,"[MAXIMUM EDUCATION , MAJOR-AUTO]","[ОБУЧЕНИЕ ПЕРСОНАЛА, ПРЯМЫЕ ПРОДАЖИ, АНАЛИТИКА...",[МОСКОВСКИЙ ГОСУДАРСТВЕННЫЙ ИНСТИТУТ МЕЖДУНАРО...,"[РУССКИЙ, АНГЛИЙСКИЙ, АРМЯНСКИЙ]"
4,Менеджер по продажам,70 000 руб.,Мужчина,30 лет,РАМЕНСКОЕ,Опыт работы 4 года 8 месяцев,"[ООО ЭВЭН , ООО ТАХОГРАФ ]",[ОПЫТ РАБОТЫ В 1С И CRM],"[МОСКОВСКИЙ БУХГАЛТЕРСКИЙ ИНСТИТУТ МБИ, МБИ ]","[РУССКИЙ, АНГЛИЙСКИЙ]"
...,...,...,...,...,...,...,...,...,...,...
821,"Менеджер, продажи",70 000 руб.,Мужчина,30 лет,САНКТ-ПЕТЕРБУРГ,Опыт работы 7 лет,"[ПОСМАС, ООО МОПМОП, ФАРТОВ, ФАРТОВ, УФССП ПО ...","[ВЛАДЕЮ ПРОГРАММОЙ 1С, ВЛАДЕЮ ПРОГРАММАМИ MICR...",[ДАЛЬНЕВОСТОЧНЫЙ ГОСУДАРСТВЕННЫЙ АГРАРНЫЙ УНИВ...,"[РУССКИЙ, АНГЛИЙСКИЙ, НЕМЕЦКИЙ]"
822,Ведущий менеджер по продажам,80 000 руб.,Мужчина,NaN,МОСКВА,Опыт работы 3 года 2 месяца,[ООО 1-ЫЙ ЛАБОРАТОРНЫЙ ЦЕНТР ЭКОБЕЗОПАСТНОСТЬ ...,"[1С, MICROSOFT OFFICE, OUTLOOK.]",[МОСКОВСКИЙ АВТОМОБИЛЬНО-ДОРОЖНЫЙ ГОСУДАРСТВЕН...,[РУССКИЙ]
823,Менеджер по продажам,40 000 руб.,Мужчина,34 года,ТОЛЬЯТТИ,Опыт работы 10 лет 9 месяцев,"[HARLEY-DAVIDSON SAMARA, ПОРШЕ ЦЕНТР МОСКВА, О...","[МОБИЛЬНОСТЬ, ВОДИТЕЛЬСКОЕ УДОСТОВЕРЕНИЕ КАТЕГ...",[ШУЙСКИЙ ГОСУДАРСТВЕННЫЙ ПЕДАГОГИЧЕСКИЙ УНИВЕР...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
824,Менеджер по продажам,40 000 руб.,Мужчина,32 года,ТИМАШЕВСК,Опыт работы 10 лет 7 месяцев,"[ООО ТОРГ-ЮГ, СНС-ЭКСПРЕСС, ООО ""ОМНИ-ЮГ"", ООО...","[МЕРЧЕНДАЙЗИНГ, РАБОТА С ТЕКУЩЕЙ БАЗОЙ КЛИЕНТО...","[ГОУ МК №5, ГОУ ПУ17, ШКОЛА ТВОРЧЕСКОЙ ФОТОГРА...","[РУССКИЙ, АНГЛИЙСКИЙ]"


Записываем данные в excel

In [129]:
writer = pd.ExcelWriter('parsing_final_dataset.xlsx', engine='xlsxwriter')
data_table.to_excel(writer, 'Sheet1')
writer.save()

In [130]:
data_table.to_csv('parsing_final_dataset.csv')

Опционально записываем данные в базу данных SQLite

In [131]:
import json

#convert lists to JSON format function:

def adapt_list_to_JSON(lst):
    return json.dumps(lst).encode('utf8')

def convert_JSON_to_list(data):
    return json.loads(data.decode('utf8'))

sqlite3.register_adapter(list, adapt_list_to_JSON)
sqlite3.register_converter("json", convert_JSON_to_list)

#Write pandas dataset to SQLite database

conn = sqlite3.connect('parsing_final_dataset.db', detect_types=sqlite3.PARSE_COLNAMES)
curs = conn.cursor
data_table.to_sql('parsing_final_dataset', conn, if_exists='replace', index=False)

Извлекаем необходимые столбцы из SQL c автоматической конвертацией из JSON обратно в списки

In [137]:
pd.read_sql('SELECT "Желаемая вакансия", "Желаемая зарплата", "Пол", "Возраст", "Местонахождения", "Общий стаж", "Работодатели" AS "Работодатели [json]", "Ключевые навыки" AS "Ключевые навыки[json]", "Образование" AS "Образование[json]", "Знание языков" AS "Знание языков[json]" FROM parsing_final_dataset', conn)

,Желаемая вакансия,Желаемая зарплата,Пол,Возраст,Местонахождения,Общий стаж,Работодатели,Ключевые навыки,Образование,Знание языков
0,Менеджер по продажам,80 000 руб.,Мужчина,35 лет,МОСКВА,Опыт работы 14 лет,"[СТИЛЬНЫЙ ПОЛ ООО, АЛИКС ГРУПП, ТОПВУД, ГК ""РО...","[ВЕДЕНИЕ КЛИЕНТСКОЙ БАЗЫ, РОЗНИЧНАЯ ТОРГОВЛЯ, ...","[МОСКОВСКИЙ ГОРОДСКОЙ ИНСТИТУТ МЕНЕДЖМЕНТА, МО...","[РУССКИЙ, АНГЛИЙСКИЙ]"
1,Менеджер по продажам,40 000 руб.,Женщина,20 лет,САНКТ-ПЕТЕРБУРГ,Опыт работы 10 месяцев,"[EXCEED TEAM, GO!ENGLISH ]","[ПОИСК И ПРИВЛЕЧЕНИЕ КЛИЕНТОВ, РЕГИОНАЛЬНЫЕ СЕ...",[ДОНСКОЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕРСИТ...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
2,Менеджер по продажам,35 000 руб.,Женщина,22 года,РОСТОВ-НА-ДОНУ,Опыт работы 2 года 4 месяца,"[АО ""ПФ ""СКБ КОНТУР"", DVA.EXPERT, ИП МАКЕЕВА Е...","[ОПЫТ ПРОДАЖ, ПРЯМЫЕ ПРОДАЖИ, ПОИСК И ПРИВЛЕЧЕ...",[ДОНСКОЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕРСИТ...,[РУССКИЙ]
3,Менеджер по продажам,90 000 руб.,Мужчина,22 года,ОДИНЦОВО,Опыт работы 1 год 5 месяцев,"[MAXIMUM EDUCATION , MAJOR-AUTO]","[ОБУЧЕНИЕ ПЕРСОНАЛА, ПРЯМЫЕ ПРОДАЖИ, АНАЛИТИКА...",[МОСКОВСКИЙ ГОСУДАРСТВЕННЫЙ ИНСТИТУТ МЕЖДУНАРО...,"[РУССКИЙ, АНГЛИЙСКИЙ, АРМЯНСКИЙ]"
4,Менеджер по продажам,70 000 руб.,Мужчина,30 лет,РАМЕНСКОЕ,Опыт работы 4 года 8 месяцев,"[ООО ЭВЭН , ООО ТАХОГРАФ ]",[ОПЫТ РАБОТЫ В 1С И CRM],"[МОСКОВСКИЙ БУХГАЛТЕРСКИЙ ИНСТИТУТ МБИ, МБИ ]","[РУССКИЙ, АНГЛИЙСКИЙ]"
...,...,...,...,...,...,...,...,...,...,...
821,"Менеджер, продажи",70 000 руб.,Мужчина,30 лет,САНКТ-ПЕТЕРБУРГ,Опыт работы 7 лет,"[ПОСМАС, ООО МОПМОП, ФАРТОВ, ФАРТОВ, УФССП ПО ...","[ВЛАДЕЮ ПРОГРАММОЙ 1С, ВЛАДЕЮ ПРОГРАММАМИ MICR...",[ДАЛЬНЕВОСТОЧНЫЙ ГОСУДАРСТВЕННЫЙ АГРАРНЫЙ УНИВ...,"[РУССКИЙ, АНГЛИЙСКИЙ, НЕМЕЦКИЙ]"
822,Ведущий менеджер по продажам,80 000 руб.,Мужчина,None,МОСКВА,Опыт работы 3 года 2 месяца,[ООО 1-ЫЙ ЛАБОРАТОРНЫЙ ЦЕНТР ЭКОБЕЗОПАСТНОСТЬ ...,"[1С, MICROSOFT OFFICE, OUTLOOK.]",[МОСКОВСКИЙ АВТОМОБИЛЬНО-ДОРОЖНЫЙ ГОСУДАРСТВЕН...,[РУССКИЙ]
823,Менеджер по продажам,40 000 руб.,Мужчина,34 года,ТОЛЬЯТТИ,Опыт работы 10 лет 9 месяцев,"[HARLEY-DAVIDSON SAMARA, ПОРШЕ ЦЕНТР МОСКВА, О...","[МОБИЛЬНОСТЬ, ВОДИТЕЛЬСКОЕ УДОСТОВЕРЕНИЕ КАТЕГ...",[ШУЙСКИЙ ГОСУДАРСТВЕННЫЙ ПЕДАГОГИЧЕСКИЙ УНИВЕР...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
824,Менеджер по продажам,40 000 руб.,Мужчина,32 года,ТИМАШЕВСК,Опыт работы 10 лет 7 месяцев,"[ООО ТОРГ-ЮГ, СНС-ЭКСПРЕСС, ООО ""ОМНИ-ЮГ"", ООО...","[МЕРЧЕНДАЙЗИНГ, РАБОТА С ТЕКУЩЕЙ БАЗОЙ КЛИЕНТО...","[ГОУ МК №5, ГОУ ПУ17, ШКОЛА ТВОРЧЕСКОЙ ФОТОГРА...","[РУССКИЙ, АНГЛИЙСКИЙ]"


**Обработка полученных данных для дальнейшего анализа**

In [134]:
dataset_imp = pd.read_sql('SELECT "Желаемая вакансия", "Желаемая зарплата", "Пол", "Возраст", "Общий стаж", "Местонахождения", "Работодатели" AS "Работодатели [json]", "Ключевые навыки" AS "Ключевые навыки[json]", "Образование" AS "Образование[json]", "Знание языков" AS "Знание языков[json]" FROM parsing_final_dataset', conn)

In [135]:
dataset_imp

,Желаемая вакансия,Желаемая зарплата,Пол,Возраст,Общий стаж,Местонахождения,Работодатели,Ключевые навыки,Образование,Знание языков
0,Менеджер по продажам,80 000 руб.,Мужчина,35 лет,Опыт работы 14 лет,МОСКВА,"[СТИЛЬНЫЙ ПОЛ ООО, АЛИКС ГРУПП, ТОПВУД, ГК ""РО...","[ВЕДЕНИЕ КЛИЕНТСКОЙ БАЗЫ, РОЗНИЧНАЯ ТОРГОВЛЯ, ...","[МОСКОВСКИЙ ГОРОДСКОЙ ИНСТИТУТ МЕНЕДЖМЕНТА, МО...","[РУССКИЙ, АНГЛИЙСКИЙ]"
1,Менеджер по продажам,40 000 руб.,Женщина,20 лет,Опыт работы 10 месяцев,САНКТ-ПЕТЕРБУРГ,"[EXCEED TEAM, GO!ENGLISH ]","[ПОИСК И ПРИВЛЕЧЕНИЕ КЛИЕНТОВ, РЕГИОНАЛЬНЫЕ СЕ...",[ДОНСКОЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕРСИТ...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
2,Менеджер по продажам,35 000 руб.,Женщина,22 года,Опыт работы 2 года 4 месяца,РОСТОВ-НА-ДОНУ,"[АО ""ПФ ""СКБ КОНТУР"", DVA.EXPERT, ИП МАКЕЕВА Е...","[ОПЫТ ПРОДАЖ, ПРЯМЫЕ ПРОДАЖИ, ПОИСК И ПРИВЛЕЧЕ...",[ДОНСКОЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИВЕРСИТ...,[РУССКИЙ]
3,Менеджер по продажам,90 000 руб.,Мужчина,22 года,Опыт работы 1 год 5 месяцев,ОДИНЦОВО,"[MAXIMUM EDUCATION , MAJOR-AUTO]","[ОБУЧЕНИЕ ПЕРСОНАЛА, ПРЯМЫЕ ПРОДАЖИ, АНАЛИТИКА...",[МОСКОВСКИЙ ГОСУДАРСТВЕННЫЙ ИНСТИТУТ МЕЖДУНАРО...,"[РУССКИЙ, АНГЛИЙСКИЙ, АРМЯНСКИЙ]"
4,Менеджер по продажам,70 000 руб.,Мужчина,30 лет,Опыт работы 4 года 8 месяцев,РАМЕНСКОЕ,"[ООО ЭВЭН , ООО ТАХОГРАФ ]",[ОПЫТ РАБОТЫ В 1С И CRM],"[МОСКОВСКИЙ БУХГАЛТЕРСКИЙ ИНСТИТУТ МБИ, МБИ ]","[РУССКИЙ, АНГЛИЙСКИЙ]"
...,...,...,...,...,...,...,...,...,...,...
821,"Менеджер, продажи",70 000 руб.,Мужчина,30 лет,Опыт работы 7 лет,САНКТ-ПЕТЕРБУРГ,"[ПОСМАС, ООО МОПМОП, ФАРТОВ, ФАРТОВ, УФССП ПО ...","[ВЛАДЕЮ ПРОГРАММОЙ 1С, ВЛАДЕЮ ПРОГРАММАМИ MICR...",[ДАЛЬНЕВОСТОЧНЫЙ ГОСУДАРСТВЕННЫЙ АГРАРНЫЙ УНИВ...,"[РУССКИЙ, АНГЛИЙСКИЙ, НЕМЕЦКИЙ]"
822,Ведущий менеджер по продажам,80 000 руб.,Мужчина,None,Опыт работы 3 года 2 месяца,МОСКВА,[ООО 1-ЫЙ ЛАБОРАТОРНЫЙ ЦЕНТР ЭКОБЕЗОПАСТНОСТЬ ...,"[1С, MICROSOFT OFFICE, OUTLOOK.]",[МОСКОВСКИЙ АВТОМОБИЛЬНО-ДОРОЖНЫЙ ГОСУДАРСТВЕН...,[РУССКИЙ]
823,Менеджер по продажам,40 000 руб.,Мужчина,34 года,Опыт работы 10 лет 9 месяцев,ТОЛЬЯТТИ,"[HARLEY-DAVIDSON SAMARA, ПОРШЕ ЦЕНТР МОСКВА, О...","[МОБИЛЬНОСТЬ, ВОДИТЕЛЬСКОЕ УДОСТОВЕРЕНИЕ КАТЕГ...",[ШУЙСКИЙ ГОСУДАРСТВЕННЫЙ ПЕДАГОГИЧЕСКИЙ УНИВЕР...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
824,Менеджер по продажам,40 000 руб.,Мужчина,32 года,Опыт работы 10 лет 7 месяцев,ТИМАШЕВСК,"[ООО ТОРГ-ЮГ, СНС-ЭКСПРЕСС, ООО ""ОМНИ-ЮГ"", ООО...","[МЕРЧЕНДАЙЗИНГ, РАБОТА С ТЕКУЩЕЙ БАЗОЙ КЛИЕНТО...","[ГОУ МК №5, ГОУ ПУ17, ШКОЛА ТВОРЧЕСКОЙ ФОТОГРА...","[РУССКИЙ, АНГЛИЙСКИЙ]"


Удаляем столбцы, которые не будем использовать, а также записи с отсутствующик значениями. По факту удаляются записи с отсутствующими ключевыми навыками.

In [138]:
dataset = dataset_imp.copy()
dataset = dataset.drop(columns=['Образование'])
dataset = dataset.drop(columns=['Желаемая вакансия'])
dataset = dataset.drop(columns=['Работодатели'])

In [143]:
dataset.dropna(inplace=True)
dataset.reset_index(drop=True, inplace=True)
dataset

,Желаемая зарплата,Пол,Возраст,Общий стаж,Местонахождения,Ключевые навыки,Знание языков
0,80 000 руб.,Мужчина,35 лет,Опыт работы 14 лет,МОСКВА,"[ВЕДЕНИЕ КЛИЕНТСКОЙ БАЗЫ, РОЗНИЧНАЯ ТОРГОВЛЯ, ...","[РУССКИЙ, АНГЛИЙСКИЙ]"
1,40 000 руб.,Женщина,20 лет,Опыт работы 10 месяцев,САНКТ-ПЕТЕРБУРГ,"[ПОИСК И ПРИВЛЕЧЕНИЕ КЛИЕНТОВ, РЕГИОНАЛЬНЫЕ СЕ...","[РУССКИЙ, АНГЛИЙСКИЙ]"
2,35 000 руб.,Женщина,22 года,Опыт работы 2 года 4 месяца,РОСТОВ-НА-ДОНУ,"[ОПЫТ ПРОДАЖ, ПРЯМЫЕ ПРОДАЖИ, ПОИСК И ПРИВЛЕЧЕ...",[РУССКИЙ]
3,90 000 руб.,Мужчина,22 года,Опыт работы 1 год 5 месяцев,ОДИНЦОВО,"[ОБУЧЕНИЕ ПЕРСОНАЛА, ПРЯМЫЕ ПРОДАЖИ, АНАЛИТИКА...","[РУССКИЙ, АНГЛИЙСКИЙ, АРМЯНСКИЙ]"
4,70 000 руб.,Мужчина,30 лет,Опыт работы 4 года 8 месяцев,РАМЕНСКОЕ,[ОПЫТ РАБОТЫ В 1С И CRM],"[РУССКИЙ, АНГЛИЙСКИЙ]"
...,...,...,...,...,...,...,...
663,40 000 руб.,Мужчина,35 лет,Опыт работы 12 лет 6 месяцев,КРАСНОДАР,[УВЕРЕННЫЙ ПОЛЬЗОВАТЕЛЬ ПК И ЗНАНИЕ ЯЗЫКОВ],"[РУССКИЙ, АНГЛИЙСКИЙ, УКРАИНСКИЙ]"
664,70 000 руб.,Мужчина,30 лет,Опыт работы 7 лет,САНКТ-ПЕТЕРБУРГ,"[ВЛАДЕЮ ПРОГРАММОЙ 1С, ВЛАДЕЮ ПРОГРАММАМИ MICR...","[РУССКИЙ, АНГЛИЙСКИЙ, НЕМЕЦКИЙ]"
665,40 000 руб.,Мужчина,34 года,Опыт работы 10 лет 9 месяцев,ТОЛЬЯТТИ,"[МОБИЛЬНОСТЬ, ВОДИТЕЛЬСКОЕ УДОСТОВЕРЕНИЕ КАТЕГ...","[РУССКИЙ, АНГЛИЙСКИЙ]"
666,40 000 руб.,Мужчина,32 года,Опыт работы 10 лет 7 месяцев,ТИМАШЕВСК,"[МЕРЧЕНДАЙЗИНГ, РАБОТА С ТЕКУЩЕЙ БАЗОЙ КЛИЕНТО...","[РУССКИЙ, АНГЛИЙСКИЙ]"


Так как ключевых навыков может быть условно "бесконечное" количество, соискатель сам их вводит без каких-либо ограничений, предлагается создать список топ ключевых навыков, которые чаще всего встречаются у всех соискателей. А затем проверяем наличия у каждого соискателя ключевыхнавыков из топ списка, если имеется - добавляем.

In [167]:
df_skills_temp=pd.DataFrame(dataset['Ключевые навыки'])
din = df_skills_temp['Ключевые навыки']
#making list of all skills
skills_list = pd.Series([item for sublist in din for item in sublist])
#count a frequency of every skill
skills_list_top =skills_list.value_counts(sort=True).rename_axis('x').reset_index(name='f')
#top 100 of all skills
v = skills_list_top.head(100)
l = din.size

ordered_top_list = v['x']
skill_temp_dict = []
skill_temp_dict_result = []
#cheking the skill from top 100 in every record
for h in range(0,l):
    xxx = din[h]
    for sk in range(0,100):
        if ordered_top_list[sk] in xxx:
            skill_temp_dict.append(ordered_top_list[sk])
            #print('yes')
        elif xxx == []:
            skill_temp_dict = np.NaN
    skill_temp_dict_result.append(skill_temp_dict)
    skill_temp_dict=[]
    data_result = {'Ключевые навыки': skill_temp_dict_result}
#making a list of skill from top 100 for every record
df_skills = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data_result.items()]))
print(df_skills)

                                       Ключевые навыки
0    [ПОЛЬЗОВАТЕЛЬ ПК, ВЕДЕНИЕ ПЕРЕГОВОРОВ, НАВЫКИ ...
1    [ПОЛЬЗОВАТЕЛЬ ПК, ВЕДЕНИЕ ПЕРЕГОВОРОВ, ПОИСК И...
2    [ПОЛЬЗОВАТЕЛЬ ПК, ПОИСК И ПРИВЛЕЧЕНИЕ КЛИЕНТОВ...
3    [ПОЛЬЗОВАТЕЛЬ ПК, ВЕДЕНИЕ ПЕРЕГОВОРОВ, ПОИСК И...
4                                                   []
..                                                 ...
663                                                 []
664  [ЗАКЛЮЧЕНИЕ ДОГОВОРОВ, ДЕЛОВАЯ ПЕРЕПИСКА, B2B ...
665  [ВЕДЕНИЕ ПЕРЕГОВОРОВ, РАБОТА В КОМАНДЕ, ВОДИТЕ...
666  [РАБОТА С ТЕКУЩЕЙ БАЗОЙ КЛИЕНТОВ, МЕРЧЕНДАЙЗИН...
667  [ПОИСК И ПРИВЛЕЧЕНИЕ КЛИЕНТОВ, АКТИВНЫЕ ПРОДАЖ...

[668 rows x 1 columns]


В некоторых резюме иногда зарплата бывает указана не в рублях, на этот случай конвентируем все в рубли:

In [162]:
from forex_python.converter import CurrencyRates

salary_converter = dataset['Желаемая зарплата'].str.split('\xa0', expand=True)

salary_converter.columns = ['value','currency','None']
salary_converter['currency'] = salary_converter['currency'].replace({'руб.': 'RUB', 'бел. руб.':'BYN'})
salary_converter['currency'] = salary_converter['currency'].str.strip()
salary_converter['value'] = salary_converter['value'].str.replace(u'\u2009','')
salary_converter['value'] = pd.to_numeric(salary_converter['value'])
ccc = CurrencyRates()
def currency_convertor(row):
    amount = row['value']
    curr = row['currency'] 
    try:
        exchange = ccc.get_rate('{}'.format(curr),'RUB')
    except:
        #for KZT
        exchange = 0.16
    new_curr = exchange * amount
    return int(new_curr)
salary_converter['In RUB'] = salary_converter.apply(currency_convertor, axis=1)

В столбце общий стаж присутсвует текст, извлекаем цифры и переводим все в десятичную дробь:

In [163]:
extract_digit = dataset.copy()
extract_digit = extract_digit['Общий стаж'].str.split(' ', expand = True)
temp_val_1 = extract_digit[2].str.split('\xa0', expand = True)
temp_val_1 = temp_val_1.rename(columns={0: "value", 1: "str"})
temp_val_1['value'] = temp_val_1['value'].astype('int64')
for i in range(len(temp_val_1)):
    if temp_val_1.loc[i,"str"]=='месяцев':
        temp_val_1.loc[i,"value"] = round(temp_val_1.loc[i,"value"]/12, 2)

temp_val_2 = extract_digit[3].str.split('\xa0', expand = True)
temp_val_2 = temp_val_2.rename(columns={0: "value", 1: "str"})
temp_val_2 = temp_val_2.replace({None: 0})
temp_val_2['value'] = temp_val_2['value'].astype('int64')
for i in range(len(temp_val_2)):
    if temp_val_2.loc[i,"str"] =='месяцев' or 'months':
        temp_val_2.loc[i,"value"] = round(temp_val_2.loc[i,"value"]/12,2)
exp_in_num = temp_val_1['value'] + temp_val_2['value']

In [164]:
exp_in_num

0      14.00
1       0.83
2       2.33
3       1.42
4       4.67
       ...  
663    12.50
664     7.00
665    10.75
666    10.58
667     3.08
Name: value, Length: 668, dtype: float64

Создаем итоговый датасет

In [165]:
dataset_result = dataset.copy()
dataset_result['Желаемая зарплата'] = salary_converter['In RUB']
dataset_result['Пол'] = dataset_result['Пол'].replace({'Женщина': 1, 'Мужчина': 0, 'Female': 1, 'Male': 0 })
age_numeric = dataset_result['Возраст'].str.split('\xa0', expand = True)
dataset_result['Возраст'] = age_numeric[0]
dataset_result['Общий стаж'] = exp_in_num
dataset_result['Местонахождения'] = pd.factorize(dataset_result['Местонахождения'])[0]
dataset_result['Местонахождения'] = dataset_result['Местонахождения'].astype("category")
dataset_result['Ключевые навыки'] = df_skills['Ключевые навыки']
key_skills_one_hot = pd.get_dummies(dataset_result['Ключевые навыки'].apply(pd.Series).stack()).sum(level=0)
dataset_result = pd.concat([dataset_result, key_skills_one_hot], axis=1)
languages_one_hot = pd.get_dummies(dataset_result['Знание языков'].apply(pd.Series).stack()).sum(level=0)
dataset_result = pd.concat([dataset_result, languages_one_hot], axis=1)
dataset_result = dataset_result.drop(columns=['Ключевые навыки'])
dataset_result = dataset_result.drop(columns=['Знание языков'])
dataset_result

,Желаемая зарплата,Пол,Возраст,Общий стаж,Местонахождения,1С-БИТРИКС,1С: ДОКУМЕНТООБОРОТ,1С: ПРЕДПРИЯТИЕ 8,1С: РОЗНИЦА,1С: ТОРГОВЛЯ,...,КОРЕЙСКИЙ,НЕМЕЦКИЙ,ПОЛЬСКИЙ,РУССКИЙ,СЕРБСКИЙ,ТАДЖИКСКИЙ,ТАТАРСКИЙ,ТУРЕЦКИЙ,УКРАИНСКИЙ,ФРАНЦУЗСКИЙ
0,80000,0,35,14.00,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,40000,1,20,0.83,1,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
2,35000,1,22,2.33,2,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
3,90000,0,22,1.42,3,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
4,70000,0,30,4.67,4,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,40000,0,35,12.50,10,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,1,0
664,70000,0,30,7.00,1,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,0,0,0,0
665,40000,0,34,10.75,76,0.0,0.0,1.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
666,40000,0,32,10.58,107,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


Полученный датесет можно уже использовать в качестве исходных данных для машинного обучения